In [23]:
import tensorflow as tf
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const_4:0", shape=(), dtype=float32) Tensor("Const_5:0", shape=(), dtype=float32)


In [24]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [25]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [26]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [27]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


In [29]:
add_and_triple = adder_node * 3.
print(sess.run([adder_node, add_and_triple], {a: 3, b: 4.5}))

[7.5, 22.5]


In [30]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

In [31]:
init = tf.global_variables_initializer()
sess.run(init)

In [32]:
print(sess.run([W, b]))

[array([ 0.30000001], dtype=float32), array([-0.30000001], dtype=float32)]


In [12]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [13]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


In [16]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


In [17]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [20]:
sess.run(init) #reset values to incorrect defaults
for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [21]:
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s, b: %s, loss: %s"%(curr_W, curr_b, curr_loss))


W: [-0.9999969], b: [ 0.99999082], loss: 5.69997e-11


In [34]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output, if placeholder is defined, then when we run the train, those value have to be provided
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to W = 0.3, b = -0.3
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


In [14]:
import numpy as np
import tensorflow as tf

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])

input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=None, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=500)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn, steps=1)
print("train metrics: %r" % train_metrics)
print("eval metrics: %r" % eval_metrics)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\BOYUAN~1.FEN\\AppData\\Local\\Temp\\tmpkot00u2f', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001E619932DA0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1 into C:\Users\BOYUAN~1.FEN\AppData\Local\Temp\tmpkot00u2f\model.ckpt.


INFO:tensorflow:loss = 22.0, step = 1


...

INFO:tensorflow:Restoring parameters from C:\Users\BOYUAN~1.FEN\AppData\Local\Temp\tmpkot00u2f\model.ckpt-500


INFO:tensorflow:Evaluation [1/1]


INFO:tensorflow:Finished evaluation at 2017-12-20-15:03:57


INFO:tensorflow:Saving dict for global step 500: average_loss = 0.0059958, global_step = 500, loss = 0.0239832


train metrics: {'average_loss': 0.00037892716, 'loss': 0.0015157086, 'global_step': 500}
eval metrics: {'average_loss': 0.0059957951, 'loss': 0.023983181, 'global_step': 500}


In [15]:
eval_metrics = estimator.evaluate(input_fn=eval_input_fn, steps=1)
print("eval metrics: %r" % eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-12-20-15:31:12


INFO:tensorflow:Restoring parameters from C:\Users\BOYUAN~1.FEN\AppData\Local\Temp\tmpkot00u2f\model.ckpt-500


INFO:tensorflow:Evaluation [1/1]


INFO:tensorflow:Finished evaluation at 2017-12-20-15:31:12


INFO:tensorflow:Saving dict for global step 500: average_loss = 0.0059958, global_step = 500, loss = 0.0239832


eval metrics: {'average_loss': 0.0059957951, 'loss': 0.023983181, 'global_step': 500}


In [21]:
eval_metrics = estimator.evaluate(input_fn=eval_input_fn, steps=500)
print("eval metrics: %r" % eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-12-20-15:44:30


INFO:tensorflow:Restoring parameters from C:\Users\BOYUAN~1.FEN\AppData\Local\Temp\tmpkot00u2f\model.ckpt-500


INFO:tensorflow:Evaluation [1/500]


INFO:tensorflow:Evaluation [2/500]


INFO:tensorflow:Evaluation [3/500]


...

INFO:tensorflow:Evaluation [499/500]

INFO:tensorflow:Evaluation [500/500]

INFO:tensorflow:Finished evaluation at 2017-12-20-15:44:34

INFO:tensorflow:Saving dict for global step 500: average_loss = 0.00599577, global_step = 500, loss = 0.0239831


eval metrics: {'average_loss': 0.0059957691, 'loss': 0.023983076, 'global_step': 500}


In [19]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1, shuffle=False)

eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("eval metrics: %r" % eval_metrics)


INFO:tensorflow:Starting evaluation at 2017-12-20-15:35:50


INFO:tensorflow:Restoring parameters from C:\Users\BOYUAN~1.FEN\AppData\Local\Temp\tmpkot00u2f\model.ckpt-500


INFO:tensorflow:Finished evaluation at 2017-12-20-15:35:50


INFO:tensorflow:Saving dict for global step 500: average_loss = 0.0059958, global_step = 500, loss = 0.0239832


eval metrics: {'average_loss': 0.0059957951, 'loss': 0.023983181, 'global_step': 500}


In [20]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("eval metrics: %r" % eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-12-20-15:36:00


INFO:tensorflow:Restoring parameters from C:\Users\BOYUAN~1.FEN\AppData\Local\Temp\tmpkot00u2f\model.ckpt-500


INFO:tensorflow:Finished evaluation at 2017-12-20-15:36:01


INFO:tensorflow:Saving dict for global step 500: average_loss = 0.00599576, global_step = 500, loss = 0.023983


eval metrics: {'average_loss': 0.0059957597, 'loss': 0.023983039, 'global_step': 500}
